# Phase 1 - Initial Exploration

Goals:
- Load data
- Gather basic info (shape, column names, data types, etc.)
- Generate summary statistics
- Identify and understand distribution of key variables
- Establish 3 questions to answer w/ data

Deliverables/Outcomes:
- Quarto slidedeck (5-10 slides) summarizing findings